In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import lxml.etree as ET
import pandas as pd
import numpy as np
import os
import glob

path = '/content/drive/My Drive/sabre-ingest/dataset'
read_files = glob.glob(os.path.join(path, '*.xml'))

In [0]:
schedule_id, stops, eticketable, departure_airport, departure_city, departure_time, departure_timezoneOffset, \
arrival_aiport, arrival_city, arrival_time, arrival_timezoneOffset, arrival_dateAdjustment, carrier_marketing, \
carrier_operating, carrier_codeShared, carrier_operatingFlightNumber, carrier_marketingFlightNumber, \
equipment_code, equipment_change, departure_date, return_date, pricing_subSource, PoSCountryCode, \
passenger_group, passenger_type, passenger_number, passenger_nonRefundable, booking_code, \
cabin_code, seats_available, availablity_break, total_price, total_taxes, currency = ([] for i in range(34))

for i in read_files:
  for event, elem in ET.iterparse(i, events = ('start',), recover = True):
    if elem.tag == 'ScheduleDesc':
      schedule_id.append(elem.get('ID'))
      stops.append(elem.get('Stops'))
      eticketable.append(elem.get('ETicketable'))
    elif elem.tag == 'Departure':
      departure_airport.append(elem.get('Airport'))
      departure_city.append(elem.get('City'))
      departure_time.append(elem.get('Time')[:8])
      departure_timezoneOffset.append(elem.get('Time')[9:])
    elif elem.tag == 'Arrival':
      arrival_aiport.append(elem.get('Airport'))
      arrival_city.append(elem.get('City'))
      arrival_time.append(elem.get('Time')[:8])
      arrival_timezoneOffset.append(elem.get('Time')[9:])
      arrival_dateAdjustment.append(elem.get('DateAdjustment'))
    elif elem.tag == 'Carrier':
      carrier_marketing.append(elem.get('Marketing'))
      carrier_operating.append(elem.get('Operating'))
      carrier_codeShared.append(elem.get('CodeShared'))
      carrier_operatingFlightNumber.append(elem.get('OperatingFlightNumber'))
      carrier_marketingFlightNumber.append(elem.get('MarketingFlightNumber'))
    elif elem.tag == 'Equipment':
      equipment_code.append(elem.get('Code'))
      equipment_change.append(elem.get('Change'))
    elif elem.tag == 'LegDescription':
      departure_date.append(elem.get('DepartureDate'))
    elif elem.tag == 'PricingInformation':
      if len(pricing_subSource) < len(schedule_id):
        pricing_subSource.append(elem.get('PricingSubsource'))
        PoSCountryCode.append(elem.get('PoSCountryCode'))
        passenger_group.append(elem.get('PassengerGroup'))
    elif elem.tag == 'PassengerInfo':
      if len(passenger_type) < len(schedule_id):  
        passenger_type.append(elem.get('PassengerType'))
        passenger_number.append(elem.get('PassengerNumber'))
        passenger_nonRefundable.append(elem.get('NonRefundable'))
    elif elem.tag == 'TotalFare':
      if len(total_price) < len(schedule_id):
        total_price.append(elem.get('TotalPrice'))
        total_taxes.append(elem.get('TotalTaxes'))
        currency.append(elem.get('Currency'))
    elif elem.tag == 'Segment':
      if len(booking_code) < len(passenger_number):
        booking_code.append(elem.get('BookingCode'))
        cabin_code.append(elem.get('CabinCode'))
        seats_available.append(elem.get('SeatsAvailable'))
        availablity_break.append(elem.get('AvailabilityBreak'))
      elem.clear()



In [0]:
# removing none values

#booking_code = [i for i in booking_code if i]
#cabin_code = [i for i in cabin_code if i]
#seats_available = [i for i in seats_available if i]
#availablity_break = [i for i in availablity_break if i]


In [0]:
return_date = departure_date[1::2]
departure_date = departure_date[::2]

In [0]:
data_dict = {
    'schedule_id': schedule_id, 'stops': stops, 'eticketable': eticketable,
    'departure_airport': departure_airport, 'departure_city': departure_city,
    'departure_time': departure_time, 'departure_timezoneOffset': departure_timezoneOffset,
    'arrival_aiport': arrival_aiport, 'arrival_city': arrival_city, 'arrival_time': arrival_time,
    'arrival_dateAdjustment': arrival_dateAdjustment, 'arrival_timezoneOffset': arrival_timezoneOffset,
    'carrier_marketing': carrier_marketing, 'carrier_operating': carrier_operating,'carrier_codeShared': carrier_codeShared,
    'carrier_operatingFlightNumber': carrier_operatingFlightNumber, 'carrier_marketingFlightNumber': carrier_marketingFlightNumber,
    'equipment_code': equipment_code, 'equipment_change': equipment_change,
    'pricing_subSource': pricing_subSource, 'PoSCountryCode': PoSCountryCode, 'passenger_group': passenger_group,
    'passenger_type': passenger_type, 'passenger_number': passenger_number, 'passenger_nonRefundable': passenger_nonRefundable,
    'total_price': total_price, 'total_taxes': total_taxes, 'currency': currency,
    'departure_date': departure_date, 'return_date': return_date,
    'booking_code': booking_code, 'cabin_code': cabin_code, 'seats_available': seats_available,
    'availablity_break': availablity_break
    }

In [0]:
df = pd.DataFrame.from_dict(data_dict, orient = 'index')

In [0]:
df = df.transpose()

In [0]:
df.to_csv('/content/drive/My Drive/sabre-ingest/sabre_ingest_data.csv')